In [24]:
from mip import Model, xsum, minimize, BINARY

qmin = [1/2, 1/2]
qmax = [1, 3/4]

wmin = [1/2, 3/4]
wmax = [1, 7/8]

ncomps = len(qmin)
I = range(ncomps)

alpha = [(wmax[i] - wmin[i])/(qmax[i] - qmin[i]) for i in I]
beta = [wmin[i] - qmin[i] * alpha[i] for i in I]

Q = 1.75

m = Model()

theta = [m.add_var(var_type=BINARY) for _ in I]
q = [m.add_var() for _ in I]

m.objective = minimize(xsum(alpha[i] * q[i] + beta[i] - beta[i] * (1 - theta[i]) for i in I))

#Constraints:
for i in I:
    m += q[i] <= qmax[i] * theta[i]
    m += q[i] >= qmin[i]* theta[i]
m += xsum(q[i] for i in I) == Q

m.optimize()


[bool(theta[i].x) for i in I], [q[i].x for i in I]

Coin0507I Presolve determined that the problem was infeasible with tolerance of 1e-08
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj 0 Primal inf 1.999999 (1)
Clp0001I Primal infeasible - objective value 2.1666667
Clp0032I PrimalInfeasible objective 2.166666667 - 4 iterations time 0.002
Starting solution of the Linear programming relaxation problem using Dual Simplex

Coin0506I Presolve 0 (-5) rows, 0 (-4) columns and 0 (-10) elements
Clp0000I Optimal - objective value 1.875
Coin0511I After Postsolve, objective 1.875, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1.875 - 0 iterations time 0.002, Presolve 0.00

Starting MIP optimization


([True, True], [1.0, 0.75])

In [12]:
alpha

[[1, 0.875]]

In [13]:
beta

[0]

In [1]:
from cosmiclib import Compressor, Optimizer

In [2]:
my_compressor = Compressor('my compressor', qmin=0.5, qmax=1, wmin=0.5, wmax=1)

In [3]:
my_optimizer = Optimizer(compressors=[my_compressor])

In [4]:
my_optimizer.setup_problem()

In [6]:
my_optimizer.beta

[0.0]

In [7]:
my_optimizer.find_opt(0)

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-3) rows, 0 (-2) columns and 0 (-5) elements
Clp0000I Optimal - objective value 0.75
Coin0511I After Postsolve, objective 0.75, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.75 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization


([True], [0.75])